# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [12]:
import pandas as pd
import requests

# Load the data from the API
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'
data = requests.get(url).json()
df = pd.DataFrame(data['items'])

# Select the PVLs performed by municipalities in the state of Rio de Janeiro
df_RJ = df[df['uf'] == 'RJ'].copy()

# Find the three most frequent statuses of the requests in the database and their frequencies
status_counts = df_RJ['status'].value_counts()
print("The three most frequent statuses of the requests are:")
print(status_counts[:3])

# Construct a new variable that contains the year of the status
df_RJ.loc[:, 'year_status'] = df_RJ['data_status'].str.slice(0, 4)

# Indicate the frequency of each year of the field constructed in item (3)
year_status_counts = df_RJ['year_status'].value_counts()
print("The frequency of each year of the status is:")
print(year_status_counts)


The three most frequent statuses of the requests are:
status
Deferido                                                 18
Arquivado                                                15
Encaminhado à PGFN com manifestação técnica favorável    10
Name: count, dtype: int64
The frequency of each year of the status is:
year_status
06/0    5
26/0    5
15/0    4
18/0    4
07/0    3
28/0    3
04/0    3
13/0    3
22/0    3
20/0    3
02/0    3
11/0    2
30/1    2
09/0    2
19/0    2
23/1    2
21/0    2
27/0    2
12/1    1
29/0    1
17/0    1
22/1    1
24/0    1
25/0    1
03/0    1
06/1    1
14/0    1
20/1    1
05/0    1
13/1    1
31/0    1
31/1    1
01/0    1
04/1    1
10/0    1
19/1    1
28/1    1
Name: count, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [17]:
# Função para obter dados da API
def get_data_from_api(uf, interested_party):
    # URL base da API
    base_url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'
    # Parâmetros da consulta
    params = {
        'uf': uf,
        'interessado': interested_party
    }
    # Fazer a requisição para a API
    response = requests.get(base_url, params=params)
    # Verificar se a requisição foi bem sucedida
    if response.status_code == 200:
        # Converter os dados JSON em um DataFrame
        data = response.json()
        df = pd.DataFrame(data['items'])
        return df
    else:
        print(f"Erro ao obter dados da API: {response.status_code}")
        return pd.DataFrame()

# 1- Obter os dados da consulta no formato DataFrame
df_RJ = get_data_from_api('RJ', 'Município')
print("1- DataFrame com os dados da consulta:\n", df_RJ.head())

# 2- Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com status em 'Arquivado por decurso de prazo' estão registradas?
df_MG = get_data_from_api('MG', 'Estado')
num_archived_requests_MG = len(df_MG[df_MG['status'] == 'Arquivado por decurso de prazo'])
print(f"\n2- O número de solicitações para o Estado de Minas Gerais com status 'Arquivado por decurso de prazo' é: {num_archived_requests_MG}")

# 3- Qual é o município da Bahia com mais solicitações deferidas?
df_BA = get_data_from_api('BA', 'Município')
most_requests_city_BA = df_BA[df_BA['status'] == 'Deferido']['interessado'].value_counts().idxmax()
print(f"\n3- O município da Bahia com mais solicitações deferidas é: {most_requests_city_BA}")

# 4- Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'
df_BA_state = get_data_from_api('BA', 'Estado')
df_BA_state.to_csv('solicitacoes_BA_Estado.csv', index=False)
print("\n4- Os dados de solicitações da Bahia, com interessado = 'Estado', foram salvos no arquivo 'solicitacoes_BA_Estado.csv'")


1- DataFrame com os dados da consulta:
    id_pleito tipo_interessado     interessado  cod_ibge  uf  \
0      63612        Município         Resende   3304201  RJ   
1      11032        Município      Petrópolis   3303906  RJ   
2      52029        Município  Miguel Pereira   3302908  RJ   
3      65489        Município     Nova Iguaçu   3303500  RJ   
4      66729        Município  Miguel Pereira   3302908  RJ   

                num_pvl              status          num_processo  \
0  PVL02.008921/2022-14  Arquivado a pedido  17944.100828/2023-12   
1                  None  Arquivado a pedido  17944.001341/2013-78   
2  PVL02.001208/2021-69  Arquivado a pedido  17944.100017/2022-22   
3  PVL02.002202/2023-71            Deferido  17944.102806/2023-89   
4  PVL02.003701/2023-85   Deferido (PVL-IF)                  None   

         data_protocolo                                      tipo_operacao  \
0  2023-08-24T17:59:04Z  Operação contratual interna (com garantia da U...   
1  2017-03